PERCOBAAN SIMULASI DETEKSI FRAUD

In [ ]:
#1. Import Library
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler

#2. Load model, scaler, dan fitur
model = joblib.load("xgb_model_smote.pkl")
scaler = joblib.load("scaler.pkl")
feature_names = joblib.load("feature_names.pkl")

#3. Cek daftar fitur agar tahu tipe apa saja yang digunakan
print("Daftar fitur yang digunakan untuk training:")
print(feature_names)

#Fungsi preprocessing input transaksi baru
def preprocess_input(data_dict):
    df = pd.DataFrame([data_dict])

    # One-hot encoding manual sesuai semua jenis type yang mungkin ada
    df['type_CASH_OUT'] = 1 if df['type'].iloc[0] == 'CASH_OUT' else 0
    df['type_DEBIT'] = 1 if df['type'].iloc[0] == 'DEBIT' else 0
    df['type_PAYMENT'] = 1 if df['type'].iloc[0] == 'PAYMENT' else 0
    df['type_TRANSFER'] = 1 if df['type'].iloc[0] == 'TRANSFER' else 0

    # Hapus kolom 'type' karena sudah di-encode
    df = df.drop('type', axis=1)

    # Isi kolom yang tidak ada dengan 0 agar cocok dengan training
    for col in feature_names:
        if col not in df.columns:
            df[col] = 0

    # Urutkan kolom agar sesuai dengan saat training
    df = df[feature_names]

    # Transformasi dengan scaler
    df_scaled = scaler.transform(df)
    return df_scaled

#5. Fungsi deteksi penipuan
def detect_fraud(input_data):
    try:
        processed = preprocess_input(input_data)
        prediction = model.predict(processed)[0]
        probability = model.predict_proba(processed)[0][1]

        status = "🔴 PENIPUAN" if prediction == 1 else "🟢 NORMAL"
        print("📌 Input:", input_data)
        print(f"✅ Hasil Deteksi: {status}")
        print(f"📊 Probabilitas Penipuan: {probability:.4f}")
    except Exception as e:
        print("❌ Error:", e)



Daftar fitur yang digunakan untuk training:
['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'isFlaggedFraud', 'type_CASH_OUT', 'type_DEBIT', 'type_PAYMENT', 'type_TRANSFER']


In [ ]:
#Contoh Input Transaksi untuk Simulasi Normal
input_transaction = {
    'step': 1,
    'amount': 200000,
    'oldbalanceOrg': 1000000,
    'newbalanceOrig': 800000,
    'oldbalanceDest': 0,
    'newbalanceDest': 200000,
    'type': 'TRANSFER'
}

detect_fraud(input_transaction)


📌 Input: {'step': 1, 'amount': 200000, 'oldbalanceOrg': 1000000, 'newbalanceOrig': 800000, 'oldbalanceDest': 0, 'newbalanceDest': 200000, 'type': 'TRANSFER'}
✅ Hasil Deteksi: 🟢 NORMAL
📊 Probabilitas Penipuan: 0.0003


In [16]:
#Contoh Input Transaksi untuk Simulasi Penipuan
input_fraud = {
    'step': 1,
    'amount': 900000,
    'oldbalanceOrg': 900000,
    'newbalanceOrig': 0,
    'oldbalanceDest': 0,
    'newbalanceDest': 0,
    'type': 'TRANSFER'
}

detect_fraud(input_fraud)


📌 Input: {'step': 1, 'amount': 900000, 'oldbalanceOrg': 900000, 'newbalanceOrig': 0, 'oldbalanceDest': 0, 'newbalanceDest': 0, 'type': 'TRANSFER'}
✅ Hasil Deteksi: 🔴 PENIPUAN
📊 Probabilitas Penipuan: 1.0000
